<a href="https://colab.research.google.com/github/waynelee9511cloud/my-colab-notebooks/blob/main/MedGemma_Training_1117.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title MedGemma Training

# ==================== CELL 1: 安裝套件 ====================
!pip install -q transformers datasets accelerate bitsandbytes peft openpyxl scikit-learn matplotlib seaborn

# ==================== CELL 2: 導入函式庫 ====================
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils import resample
from google.colab import files, drive
import os
import json
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
print('✅ 所有函式庫導入完成！')

# ==================== CELL 3: 掛載 Google Drive ====================
print('📁 正在掛載 Google Drive...')
drive.mount('/content/drive')
print('✅ Google Drive 掛載完成！')

# ==================== CELL 4: 上傳檔案 ====================
print('\n📤 請上傳您的訓練檔案')
print('  1. Excel 訓練資料檔案')
print('  2. 台灣藥物商品名稱 txt 檔案（選填）')
uploaded = files.upload()

excel_file = None
drug_names_file = None

for filename in uploaded.keys():
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        excel_file = filename
        print(f'✅ Excel 檔案: {excel_file}')
    elif filename.endswith('.txt'):
        drug_names_file = filename
        print(f'✅ 藥物名稱檔案: {drug_names_file}')

if not excel_file:
    raise ValueError('❌ 未找到 Excel 檔案！')

# ==================== CELL 5: 讀取台灣藥物名稱 ====================
taiwan_drug_names = set()

if drug_names_file:
    print(f'\n📋 正在讀取台灣藥物商品名稱...')
    try:
        with open(drug_names_file, 'r', encoding='utf-8') as f:
            for line in f:
                drug_name = line.strip()
                if drug_name:
                    taiwan_drug_names.add(drug_name)
        print(f'✅ 成功讀取 {len(taiwan_drug_names)} 個台灣藥物商品名稱')
        print(f'   範例: {list(taiwan_drug_names)[:5]}')
    except Exception as e:
        print(f'⚠️ 讀取藥物名稱檔案時發生錯誤: {e}')
else:
    print('\n⚠️ 未提供台灣藥物名稱檔案')

# ==================== CELL 6: 讀取 Excel 資料 ====================
print('\n📊 正在讀取 Excel 檔案...')

excel_data = pd.ExcelFile(excel_file)
sheet_names = excel_data.sheet_names
print(f'✅ 發現 {len(sheet_names)} 個工作表: {sheet_names}')

all_dataframes = []

for sheet_name in sheet_names:
    df_sheet = pd.read_excel(excel_data, sheet_name=sheet_name)

    required_columns = ['Term_To_Check', 'Correct_Output', 'category']
    missing_columns = [col for col in required_columns if col not in df_sheet.columns]

    if missing_columns:
        print(f'⚠️ 工作表 "{sheet_name}" 缺少欄位: {missing_columns}')
        continue

    total = len(df_sheet)
    correct = len(df_sheet[df_sheet['Correct_Output'] == 'No issues found.'])
    error = len(df_sheet[df_sheet['Correct_Output'] != 'No issues found.'])

    print(f'  📄 {sheet_name}: {total} 筆 (錯誤: {error} / {error/total*100:.1f}%)')
    all_dataframes.append(df_sheet)

df = pd.concat(all_dataframes, ignore_index=True)

print(f'\n✅ 資料合併完成')
print(f'  總筆數: {len(df)}')
print(f'  正確術語: {len(df[df["Correct_Output"] == "No issues found."])} 筆')
print(f'  錯誤術語: {len(df[df["Correct_Output"] != "No issues found."])} 筆')

# ==================== CELL 7: 整合台灣藥物名稱 ====================
if taiwan_drug_names and len(taiwan_drug_names) > 0:
    print('\n' + '=' * 80)
    print('🇹🇼 整合台灣藥物商品名稱到訓練資料')
    print('=' * 80)

    drug_training_data = []
    for drug_name in taiwan_drug_names:
        drug_training_data.append({
            'Term_To_Check': drug_name,
            'Correct_Output': 'No issues found.',
            'category': 'CM'
        })

    df_drugs = pd.DataFrame(drug_training_data)
    original_len = len(df)
    df = pd.concat([df, df_drugs], ignore_index=True)

    print(f'\n✅ 成功整合 {len(df_drugs)} 個台灣藥物名稱')
    print(f'   原始資料: {original_len} 筆')
    print(f'   整合後: {len(df)} 筆')
    print('=' * 80)

# ==================== CELL 8: 資料平衡 ====================
print('\n' + '=' * 80)
print('📊 資料平衡處理')
print('=' * 80)

TARGET_ERROR_RATIO = 0.40
MAX_TOTAL_SAMPLES = 3500

current_error_count = len(df[df['Correct_Output'] != 'No issues found.'])
current_correct_count = len(df[df['Correct_Output'] == 'No issues found.'])
current_ratio = current_error_count / len(df)

print(f'\n📈 資料分布:')
print(f'  總筆數: {len(df)}')
print(f'  正確術語: {current_correct_count} ({current_correct_count/len(df)*100:.1f}%)')
print(f'  錯誤術語: {current_error_count} ({current_ratio*100:.1f}%)')

if current_ratio < TARGET_ERROR_RATIO:
    print(f'\n⚠️ 錯誤比例不足，開始過採樣...')

    df_correct = df[df['Correct_Output'] == 'No issues found.'].copy()
    df_error = df[df['Correct_Output'] != 'No issues found.'].copy()

    needed_error_count = int(TARGET_ERROR_RATIO * len(df_correct) / (1 - TARGET_ERROR_RATIO))
    potential_total = len(df_correct) + needed_error_count

    if potential_total > MAX_TOTAL_SAMPLES:
        target_error_count = int(MAX_TOTAL_SAMPLES * TARGET_ERROR_RATIO)
        target_correct_count = MAX_TOTAL_SAMPLES - target_error_count

        if len(df_correct) > target_correct_count:
            df_correct = resample(df_correct, replace=False, n_samples=target_correct_count, random_state=42)

        df_error_upsampled = resample(df_error, replace=True, n_samples=target_error_count, random_state=42)
    else:
        df_error_upsampled = resample(df_error, replace=True, n_samples=needed_error_count, random_state=42)

    df = pd.concat([df_correct, df_error_upsampled], ignore_index=True)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    print(f'\n🎉 資料平衡完成！')
    print(f'  總筆數: {len(df)}')

    del df_correct, df_error, df_error_upsampled
    gc.collect()

print('=' * 80)

# ==================== CELL 9: 設定參數 ====================
BASE_MODEL_ID = 'google/medgemma-4b-it'
LEARNING_RATE = 1e-4
NUM_EPOCHS = 3
BATCH_SIZE = 2
MAX_LENGTH = 384
GRADIENT_ACCUMULATION_STEPS = 8
N_SPLITS = 5

print(f'\n⚙️ 訓練參數設定')
print('=' * 50)
print(f'使用模型: {BASE_MODEL_ID}')
print(f'交叉驗證折數: {N_SPLITS}-fold')
print(f'學習率: {LEARNING_RATE}')
print(f'訓練輪數: {NUM_EPOCHS}')
print(f'批次大小: {BATCH_SIZE}')
print('=' * 50)

# ==================== CELL 10: 準備訓練資料格式 ====================
def create_prompt(term):
    return f'Please check if there is any error in this medical term: {term}'

def format_example(term, response, category):
    prompt = create_prompt(term)
    formatted_text = f'<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n{response}<end_of_turn>'
    return {
        'text': formatted_text,
        'term': term,
        'response': response,
        'category': category
    }

all_examples = []
for idx, row in df.iterrows():
    example = format_example(
        term=row['Term_To_Check'],
        response=row['Correct_Output'],
        category=row['category']
    )
    all_examples.append(example)

print(f'✅ 準備了 {len(all_examples)} 個訓練範例')

# ==================== CELL 11: 初始化交叉驗證 ====================
print('\n' + '=' * 80)
print('🔄 開始 5-Fold 交叉驗證')
print('=' * 80)

fold_results = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'confusion_matrices': [],
    'category_results': []
}

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
data_indices = list(range(len(all_examples)))

print(f'✅ 初始化完成')

# ==================== 接下來的 CELL 12-16: 每個 Fold 的訓練 ====================
# 注意：由於每個 fold 的訓練程式碼很長，建議將下面的程式碼分成 5 個獨立的 cell
# 每個 cell 訓練一個 fold，這樣可以在任何一個 fold 失敗時不影響其他 fold

# 以下是通用的訓練循環程式碼，您可以複製 5 次並修改 fold_idx

# ==================== CELL 12-16: Fold 訓練循環 (請分成5個cell執行) ====================
# 將以下程式碼複製5次，每次修改 fold_idx 的值(0, 1, 2, 3, 4)

for fold_idx, (train_indices, test_indices) in enumerate(kfold.split(data_indices)):
    print(f'\n' + '=' * 80)
    print(f'📊 Fold {fold_idx + 1}/{N_SPLITS}')
    print('=' * 80)

    # 分割資料
    train_examples = [all_examples[i] for i in train_indices]
    test_examples = [all_examples[i] for i in test_indices]

    print(f'  訓練集大小: {len(train_examples)}')
    print(f'  測試集大小: {len(test_examples)}')

    # 載入模型
    print(f'\n🔧 Fold {fold_idx + 1}: 載入模型...')
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )

    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'right'

    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_ID,
        quantization_config=bnb_config,
        device_map='auto',
        trust_remote_code=True
    )

    print('✅ 模型載入完成')

    # 配置 LoRA
    print(f'\n🔧 Fold {fold_idx + 1}: 配置 LoRA...')
    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
        task_type='CAUSAL_LM',
        bias='none'
    )

    model = get_peft_model(model, lora_config)
    print('✅ LoRA 配置完成')

    # 準備訓練資料
    print(f'\n🔄 Fold {fold_idx + 1}: 準備訓練資料...')

    def tokenize_function(examples):
        texts = examples['text'] if isinstance(examples['text'], list) else [examples['text']]
        tokenized = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors=None)
        tokenized['labels'] = tokenized['input_ids'].copy()
        return tokenized

    train_dataset_dict = {
        'text': [ex['text'] for ex in train_examples],
        'term': [ex['term'] for ex in train_examples],
        'response': [ex['response'] for ex in train_examples],
        'category': [ex['category'] for ex in train_examples]
    }

    test_dataset_dict = {
        'text': [ex['text'] for ex in test_examples],
        'term': [ex['term'] for ex in test_examples],
        'response': [ex['response'] for ex in test_examples],
        'category': [ex['category'] for ex in test_examples]
    }

    train_dataset = Dataset.from_dict(train_dataset_dict)
    test_dataset = Dataset.from_dict(test_dataset_dict)

    train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
    test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)

    print('✅ 資料準備完成')
    torch.cuda.empty_cache()
    gc.collect()

    # 設定訓練參數
    output_dir = f'/content/drive/MyDrive/medgemma_cv_fold_{fold_idx + 1}'
    os.makedirs(output_dir, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        lr_scheduler_type='cosine',
        warmup_ratio=0.1,
        eval_strategy='steps',
        eval_steps=100,
        save_strategy='steps',
        save_steps=100,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model='eval_loss',
        greater_is_better=False,
        logging_steps=20,
        logging_dir=f'{output_dir}/logs',
        push_to_hub=False,
        report_to='none',
        fp16=True,
        remove_unused_columns=False,
        seed=42 + fold_idx,
        data_seed=42 + fold_idx,
        optim='paged_adamw_8bit',
        weight_decay=0.01,
        max_grad_norm=1.0,
        gradient_checkpointing=True,
        dataloader_num_workers=0,
    )

    # 訓練模型
    print(f'\n🚀 Fold {fold_idx + 1}: 開始訓練...')

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    try:
        train_result = trainer.train()
        print(f'\n✅ Fold {fold_idx + 1} 訓練完成')
        trainer.save_model(f'{output_dir}/final_model')
        tokenizer.save_pretrained(f'{output_dir}/final_model')
    except RuntimeError as e:
        if 'out of memory' in str(e):
            print(f'\n❌ Fold {fold_idx + 1}: Out of Memory 錯誤')
        continue

    # 評估模型
    print(f'\n📊 Fold {fold_idx + 1}: 評估測試集...')

    model.eval()
    predictions = []
    ground_truths = []
    categories = []

    for i, example in enumerate(test_examples):
        if i % 50 == 0:
            print(f'  進度: {i}/{len(test_examples)}')

        term = example['term']
        true_response = example['response']
        category = example['category']

        prompt = create_prompt(term)
        formatted = f'<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n'

        inputs = tokenizer(formatted, return_tensors='pt').to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True
            )

        pred_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()

        pred_label = 1 if ('incorrect' in pred_text.lower() or 'error' in pred_text.lower()) else 0
        true_label = 0 if true_response == 'No issues found.' else 1

        predictions.append(pred_label)
        ground_truths.append(true_label)
        categories.append(category)

    # 計算指標
    accuracy = accuracy_score(ground_truths, predictions)
    precision = precision_score(ground_truths, predictions, zero_division=0)
    recall = recall_score(ground_truths, predictions, zero_division=0)
    f1 = f1_score(ground_truths, predictions, zero_division=0)
    cm = confusion_matrix(ground_truths, predictions)

    fold_results['accuracy'].append(accuracy)
    fold_results['precision'].append(precision)
    fold_results['recall'].append(recall)
    fold_results['f1'].append(f1)
    fold_results['confusion_matrices'].append(cm)

    print(f'\n📈 Fold {fold_idx + 1} 評估結果:')
    print(f'  Accuracy:  {accuracy:.4f}')
    print(f'  Precision: {precision:.4f}')
    print(f'  Recall:    {recall:.4f}')
    print(f'  F1 Score:  {f1:.4f}')

    # 各類別性能分析
    category_metrics = {}
    unique_categories = set(categories)

    for cat in unique_categories:
        cat_indices = [i for i, c in enumerate(categories) if c == cat]
        cat_predictions = [predictions[i] for i in cat_indices]
        cat_ground_truths = [ground_truths[i] for i in cat_indices]

        if len(cat_indices) > 0:
            cat_accuracy = accuracy_score(cat_ground_truths, cat_predictions)
            cat_precision = precision_score(cat_ground_truths, cat_predictions, zero_division=0)
            cat_recall = recall_score(cat_ground_truths, cat_predictions, zero_division=0)
            cat_f1 = f1_score(cat_ground_truths, cat_predictions, zero_division=0)

            category_metrics[cat] = {
                'accuracy': cat_accuracy,
                'precision': cat_precision,
                'recall': cat_recall,
                'f1': cat_f1,
                'samples': len(cat_indices)
            }

    fold_results['category_results'].append(category_metrics)

    # 清理記憶體
    del model, trainer, train_dataset, test_dataset
    torch.cuda.empty_cache()
    gc.collect()

    print(f'\n✅ Fold {fold_idx + 1} 完成！')
    print('=' * 80)

# ==================== CELL 17: 統計分析 ====================
print('\n' + '=' * 80)
print('📊 交叉驗證統計分析')
print('=' * 80)

metrics_stats = {}
for metric in ['accuracy', 'precision', 'recall', 'f1']:
    values = fold_results[metric]
    mean = np.mean(values)
    std = np.std(values, ddof=1)

    confidence_level = 0.95
    degrees_freedom = len(values) - 1
    confidence_interval = stats.t.interval(confidence_level, degrees_freedom, mean, stats.sem(values))

    metrics_stats[metric] = {
        'mean': mean,
        'std': std,
        'ci_lower': confidence_interval[0],
        'ci_upper': confidence_interval[1],
        'values': values
    }

    print(f'\n{metric.upper()}:')
    print(f'  各 Fold 結果: {[f"{v:.4f}" for v in values]}')
    print(f'  平均值: {mean:.4f}')
    print(f'  標準差: {std:.4f}')
    print(f'  95% 信賴區間: [{confidence_interval[0]:.4f}, {confidence_interval[1]:.4f}]')

# ==================== CELL 18: 混淆矩陣視覺化 ====================
avg_cm = np.mean(fold_results['confusion_matrices'], axis=0)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('5-Fold Cross Validation - Confusion Matrices', fontsize=16, fontweight='bold')

for idx in range(N_SPLITS):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]

    cm = fold_results['confusion_matrices'][idx]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Correct', 'Error'],
                yticklabels=['Correct', 'Error'])
    ax.set_title(f'Fold {idx + 1}\nF1={fold_results["f1"][idx]:.4f}')
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')

ax = axes[1, 2]
sns.heatmap(avg_cm, annot=True, fmt='.1f', cmap='Greens', ax=ax,
            xticklabels=['Correct', 'Error'],
            yticklabels=['Correct', 'Error'])
ax.set_title(f'Average\nF1={metrics_stats["f1"]["mean"]:.4f}±{metrics_stats["f1"]["std"]:.4f}')

plt.tight_layout()
output_path = '/content/drive/MyDrive/confusion_matrices_cv.png'
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f'\n✅ 混淆矩陣已保存至: {output_path}')
plt.show()

# ==================== CELL 19: 性能指標視覺化 ====================
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('5-Fold Cross Validation - Performance Metrics', fontsize=16, fontweight='bold')

metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

for idx, (metric, name) in enumerate(zip(metrics_to_plot, metric_names)):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]

    values = metrics_stats[metric]['values']
    mean = metrics_stats[metric]['mean']
    ci_lower = metrics_stats[metric]['ci_lower']
    ci_upper = metrics_stats[metric]['ci_upper']

    folds = list(range(1, N_SPLITS + 1))
    ax.plot(folds, values, marker='o', linewidth=2, markersize=8, label='Fold Results')
    ax.axhline(y=mean, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean:.4f}')
    ax.axhspan(ci_lower, ci_upper, alpha=0.2, color='red', label=f'95% CI')

    ax.set_xlabel('Fold', fontsize=12)
    ax.set_ylabel(name, fontsize=12)
    ax.set_title(f'{name}\nMean: {mean:.4f} ± {metrics_stats[metric]["std"]:.4f}', fontsize=12)
    ax.set_xticks(folds)
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_ylim([min(0.9, min(values) - 0.02), 1.0])

plt.tight_layout()
output_path = '/content/drive/MyDrive/performance_metrics_cv.png'
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f'\n✅ 性能指標圖已保存至: {output_path}')
plt.show()

# ==================== CELL 20: 生成詳細報告 ====================
report_lines = []
report_lines.append('=' * 80)
report_lines.append('MedGemma 醫療術語校正模型 - 5-Fold 交叉驗證報告')
report_lines.append('=' * 80)
report_lines.append('')

report_lines.append('【基本資訊】')
report_lines.append(f'  模型: {BASE_MODEL_ID}')
report_lines.append(f'  訓練資料總數: {len(all_examples)}')
report_lines.append(f'  交叉驗證折數: {N_SPLITS}')
if taiwan_drug_names:
    report_lines.append(f'  台灣藥物名稱數量: {len(taiwan_drug_names)}')
report_lines.append('')

report_lines.append('【整體性能統計】')
for metric, name in zip(['accuracy', 'precision', 'recall', 'f1'],
                       ['Accuracy', 'Precision', 'Recall', 'F1 Score']):
    stats_data = metrics_stats[metric]
    report_lines.append(f'\n{name}:')
    report_lines.append(f'  平均值: {stats_data["mean"]:.4f}')
    report_lines.append(f'  標準差: {stats_data["std"]:.4f}')
    report_lines.append(f'  95% 信賴區間: [{stats_data["ci_lower"]:.4f}, {stats_data["ci_upper"]:.4f}]')

report_lines.append('\n' + '=' * 80)

report_text = '\n'.join(report_lines)
report_path = '/content/drive/MyDrive/cv_report.txt'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report_text)

print(f'\n✅ 詳細報告已保存至: {report_path}')
print('\n報告內容:')
print(report_text)

print('\n' + '=' * 80)
print('🎉 5-Fold 交叉驗證完成！')
print('=' * 80)
print(f'\n📊 最終結果摘要:')
print(f'  F1 Score:  {metrics_stats["f1"]["mean"]:.4f} ± {metrics_stats["f1"]["std"]:.4f}')
print(f'  Accuracy:  {metrics_stats["accuracy"]["mean"]:.4f} ± {metrics_stats["accuracy"]["std"]:.4f}')